# Rhodium: Circular Arithmetic for Geographic Coordinates

This notebook demonstrates how rhodium solves common bugs in geographic coordinate calculations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marszdf/rhodium/blob/main/rhodium_demo.ipynb)

## Installation

In [ ]:
# Install rhodium and dependencies for visualization
!pip install elemental-rhodium matplotlib numpy

## Setup Visualizations
We use a helper module to keep the notebook clean.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Wedge, Rectangle

def setup_style():
    """Configure matplotlib for cleaner visualizations."""
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['figure.dpi'] = 100
    plt.rcParams['axes.titlesize'] = 14
    plt.rcParams['axes.labelsize'] = 12

def plot_bearing_comparison(b1, b2, naive_diff, correct_diff):
    """
    Visualize the difference between naive and correct bearing arithmetic.
    """
    fig = plt.figure(figsize=(14, 7))
    
    # Setup Polar Axes
    ax1 = fig.add_subplot(121, projection='polar')
    ax2 = fig.add_subplot(122, projection='polar')
    
    for ax, title, is_correct in [(ax1, "Naive Arithmetic", False), (ax2, "Rhodium Arithmetic", True)]:
        ax.set_theta_zero_location('N')
        ax.set_theta_direction(-1)
        ax.set_title(title, pad=20, size=16, weight='bold', color='green' if is_correct else 'darkred')
        ax.set_yticklabels([])
        
        # Draw compass points
        ax.set_xticks(np.radians([0, 90, 180, 270]))
        ax.set_xticklabels(['N', 'E', 'S', 'W'])
        
        # Plot Bearings (Arrows)
        for angle, label, color in [(b1, 'Start', 'blue'), (b2, 'End', 'purple')]:
            rad = np.radians(angle)
            ax.arrow(rad, 0, 0, 0.9, alpha=0.5, width=0.05, 
                     edgecolor=color, facecolor=color, lw=2, zorder=5)
            
            # Label
            ax.text(rad, 1.1, f"{label}\n{angle}°", ha='center', va='center', 
                    weight='bold', color=color)

    # --- Plot Naive Path (Left) ---
    # Naive assumes linear difference: e.g., 5 - 355 = -350
    # It wraps the "long way" around the circle
    if abs(naive_diff) > 180:
        # Long way
        start = b1
        end = b1 + naive_diff
        if naive_diff < 0: # Counter-clockwise
            theta = np.linspace(np.radians(start), np.radians(end), 100)
        else:
            theta = np.linspace(np.radians(start), np.radians(end), 100)
    else:
        # Short way (coincidentally correct in some cases, but conceptually linear)
        theta = np.linspace(np.radians(b1), np.radians(b1 + naive_diff), 100)
        
    ax1.plot(theta, [0.7]*len(theta), color='red', lw=4, alpha=0.6, label='Calculated Path')
    ax1.text(0.5, -0.1, f"Result: {naive_diff}°\n(Wrong Direction/Magnitude)", 
             transform=ax1.transAxes, ha='center', color='darkred', weight='bold')

    # --- Plot Rhodium Path (Right) ---
    # Always shortest path
    if correct_diff > 0: # Clockwise
        theta = np.linspace(np.radians(b1), np.radians(b1 + correct_diff), 100)
    else: # Counter-clockwise
        theta = np.linspace(np.radians(b1), np.radians(b1 + correct_diff), 100)
        
    ax2.plot(theta, [0.7]*len(theta), color='green', lw=4, alpha=0.6, label='Shortest Path')
    ax2.text(0.5, -0.1, f"Result: {correct_diff}°\n(Correct Shortest Arc)", 
             transform=ax2.transAxes, ha='center', color='green', weight='bold')

    plt.tight_layout()
    plt.show()

def plot_antimeridian_problem(p1, p2, naive_mid, correct_mid):
    """
    Visualize the antimeridian crossing problem on a flat map.
    """
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Setup Map styling
    ax.set_aspect('equal')
    ax.set_xlim(-185, 185)
    ax.set_ylim(-90, 90)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # Draw Background zones
    ax.axvspan(-180, -20, color='blue', alpha=0.05) # Pacific/Americas approx
    ax.axvspan(20, 180, color='blue', alpha=0.05)   # Asia/Pacific approx
    
    # Draw Antimeridian
    ax.axvline(180, color='black', linestyle='-', lw=1)
    ax.axvline(-180, color='black', linestyle='-', lw=1)
    ax.text(180, 92, "Antimeridian\n(±180°)", ha='center', fontweight='bold')
    
    # Plot Cities
    ax.plot(p1.lng, p1.lat, 'o', color='black', ms=8, label='Cities')
    ax.text(p1.lng + 2, p1.lat, "Tokyo\n(139.7°)", va='center')
    
    ax.plot(p2.lng, p2.lat, 'o', color='black', ms=8)
    ax.text(p2.lng - 2, p2.lat, "San Francisco\n(-122.4°)", ha='right', va='center')
    
    # --- Naive Path (Linear) ---
    ax.plot([p1.lng, p2.lng], [p1.lat, p2.lat], 
            color='red', linestyle='--', lw=2, label='Naive Linear Path')
    
    # Naive Midpoint
    ax.plot(naive_mid.lng, naive_mid.lat, 'X', color='red', ms=12, label='Naive Midpoint')
    ax.text(naive_mid.lng, naive_mid.lat - 5, "WRONG\n(Atlantic Ocean)", 
            color='darkred', ha='center', fontweight='bold')
            
    # --- Correct Path (Crosses Date Line) ---
    # We need to draw this in two segments for matplotlib: Tokyo -> 180 and -180 -> SF
    
    # Segment 1: Tokyo to 180
    lat_at_180 = p1.lat + (p2.lat - p1.lat) * (180 - p1.lng) / ( (180 + p2.lng + 360) - p1.lng ) # rough interp
    # Actually just visually connecting to edge is enough for demo
    ax.plot([p1.lng, 180], [p1.lat, (p1.lat+p2.lat)/2], color='green', lw=3, alpha=0.6)
    ax.plot([-180, p2.lng], [(p1.lat+p2.lat)/2, p2.lat], color='green', lw=3, alpha=0.6, label='Rhodium Path')
    
    # Correct Midpoint
    # It might be around 170 or -170.
    ax.plot(correct_mid.lng, correct_mid.lat, 'P', color='green', ms=14, label='Rhodium Midpoint')
    ax.text(correct_mid.lng, correct_mid.lat + 5, "CORRECT\n(Pacific Ocean)", 
            color='darkgreen', ha='center', fontweight='bold')

    ax.set_title("The Pacific Ocean Problem: Naive vs Rhodium Averaging", size=16, pad=20)
    ax.legend(loc='lower center', ncol=2, frameon=True, fontsize=11)
    
    plt.tight_layout()
    plt.show()

def plot_bbox_fixed(fiji_box, points_in, points_out):
    """Visualize a bounding box crossing the antimeridian."""
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Setup Map
    ax.set_xlim(160, -160 + 360) # Show 160E to 160W continuously
    # We need to shift negative longitudes to be > 180 for this view
    
    def shift(lng):
        return lng + 360 if lng < 0 else lng
        
    ax.set_ylim(-40, 0)
    ax.set_xlabel("Longitude (Shifted view: 160° to 200°)")
    ax.set_ylabel("Latitude")
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # Draw Antimeridian
    ax.axvline(180, color='gray', linestyle='--', lw=2)
    ax.text(180, 0.5, "Antimeridian (180°)", ha='center', transform=ax.get_xaxis_transform())

    # Plot BBox
    # Fiji: West 177, East -178.
    # In shifted view: West 177, East 182 (-178 + 360)
    w = shift(fiji_box.west)
    e = shift(fiji_box.east)
    
    rect = Rectangle((w, fiji_box.south), e - w, fiji_box.north - fiji_box.south,
                     linewidth=2, edgecolor='green', facecolor='lightgreen', alpha=0.3,
                     label='Rhodium BBox (Correctly wraps)')
    ax.add_patch(rect)
    
    # Plot Points
    for p, name in points_in:
        s_lng = shift(p.lng)
        ax.plot(s_lng, p.lat, 'o', color='darkgreen', ms=8)
        ax.text(s_lng + 0.5, p.lat, f"{name}\n(Inside)", color='darkgreen', va='center')
        
    for p, name in points_out:
        # Sydney is 151. Don't shift.
        # But wait, 151 is outside our view range [160, 200]?
        # Let's adjust view or points. Sydney is far west.
        s_lng = shift(p.lng)
        ax.plot(s_lng, p.lat, 'x', color='red', ms=8)
        ax.text(s_lng + 0.5, p.lat, f"{name}\n(Outside)", color='red', va='center')

    ax.set_title("Bounding Boxes across the Antimeridian", size=16)
    ax.legend()
    plt.tight_layout()
    plt.show()

setup_style()


## The Problem: Naive Arithmetic Fails at Boundaries

Geographic coordinates wrap around at boundaries (360° for bearings, ±180° for longitude). Regular arithmetic produces wrong results.

In [ ]:
# Example 1: Aircraft heading change
heading_1 = 355  # degrees (almost north)
heading_2 = 5    # degrees (just past north)

# WRONG: Naive arithmetic
naive_diff = heading_2 - heading_1
print(f"❌ Naive difference: {naive_diff}° (WRONG! Says turned 350° left)")

# CORRECT: Using rhodium
from rhodium import bearing
correct_diff = bearing.diff(heading_1, heading_2)
print(f"✅ Rhodium difference: {correct_diff}° (Correct! Turned 10° right)")

In [ ]:
# Example 2: Pacific Ocean longitude averaging
tokyo_lng = 139.7      # Japan
san_francisco_lng = -122.4  # USA

# WRONG: Naive average
naive_avg = (tokyo_lng + san_francisco_lng) / 2
print(f"❌ Naive average: {naive_avg}° (WRONG! That's in the Atlantic Ocean)")

# CORRECT: Using rhodium
from rhodium import lng
correct_avg = lng.mean([tokyo_lng, san_francisco_lng])
print(f"✅ Rhodium average: {correct_avg}° (Correct! In the Pacific Ocean)")

## Visualization 1: Bearing Arithmetic on a Compass Rose

In [ ]:
plot_bearing_comparison(
    b1=heading_1, 
    b2=heading_2, 
    naive_diff=naive_diff, 
    correct_diff=correct_diff
)

## Visualization 2: Longitude Averaging Across the Antimeridian

In [ ]:
from rhodium.bbox import Point

p1 = Point(lng=139.7, lat=35.7)   # Tokyo
p2 = Point(lng=-122.4, lat=37.8)  # San Francisco

naive_mid = Point(lng=naive_avg, lat=(p1.lat+p2.lat)/2)
correct_mid = Point(lng=correct_avg, lat=(p1.lat+p2.lat)/2)

plot_antimeridian_problem(p1, p2, naive_mid, correct_mid)

## Bearing Arithmetic API

All operations handle wraparound correctly at 360°.

In [ ]:
from rhodium import bearing

# Normalize to [0, 360)
print(bearing.normalize(720))    # 0.0
print(bearing.normalize(-45))    # 315.0

# Shortest angular difference
print(bearing.diff(10, 350))     # -20.0 (not 340)
print(bearing.diff(350, 10))     # 20.0 (not -340)

# Average bearing
print(bearing.mean([350, 10]))     # 0.0 (crosses north)

# Interpolation
print(bearing.interpolate(350, 10, 0.5))  # 0.0 (halfway)

# Opposite bearing
print(bearing.opposite(45))      # 225.0

# Check if within tolerance
print(bearing.within(5, 355, tolerance=15))  # True

## Longitude Arithmetic API

All operations handle the antimeridian (±180°) correctly.

In [ ]:
from rhodium import lng

# Normalize to (-180, 180]
print(lng.normalize(190))        # -170.0
print(lng.normalize(-190))       # 170.0

# Shortest difference
print(lng.diff(170, -170))       # -20.0 (crosses antimeridian)

# Average (handles Pacific Ocean)
print(lng.mean([170, -170]))       # 180.0
print(lng.mean([139.7, -122.4]))   # Approximately -171.35 (Pacific)

# Interpolation
print(lng.interpolate(170, -170, 0.5))  # 180.0

## Latitude Operations API

Latitude has hard boundaries at ±90°.

In [ ]:
from rhodium import lat

# Clamp to [-90, 90]
print(lat.clamp(100))            # 90.0
print(lat.clamp(-100))           # -90.0

# Validation
print(lat.is_valid(45))          # True
print(lat.is_valid(100))         # False

# Midpoint
print(lat.midpoint(10, 50))      # 30.0

# Hemisphere
print(lat.hemisphere(45))        # 'N'
print(lat.hemisphere(-33.9))     # 'S'

# Check if within range
print(lat.within(45, 40, 50))    # True

## Bounding Boxes: The Alaska/Fiji Problem

Regular bounding boxes fail when crossing the antimeridian.

In [ ]:
from rhodium import bbox
from rhodium.bbox import Point, BBox

# Fiji crosses the antimeridian (west=177°, east=-178°)
fiji = BBox(west=177, east=-178, south=-19, north=-16)

print(f"Fiji width: {bbox.width(fiji)}° (not negative!)")
print(f"Fiji height: {bbox.height(fiji)}°")
print(f"Crosses antimeridian: {bbox.crosses_antimeridian(fiji)}")

# Check if point is in Fiji
suva = Point(lng=178.4, lat=-18.1)  # Suva, Fiji's capital
print(f"Suva in Fiji bbox: {bbox.contains(fiji, suva)}")

## Visualization 3: Bounding Box with Antimeridian Crossing

In [ ]:
# Define points to check
points_in = [
    (Point(lng=178.4, lat=-18.1), "Suva"),
    (Point(lng=-179.9, lat=-16.5), "Taveuni")
]
points_out = [
    (Point(lng=170.0, lat=-20.0), "Ocean (West)"),
    (Point(lng=-170.0, lat=-20.0), "Ocean (East)")
]

plot_bbox_fixed(fiji, points_in, points_out)

## Bounding Box API

In [ ]:
from rhodium import bbox
from rhodium.bbox import Point, BBox

# Create from corner points
sw = Point(lng=-122.5, lat=37.7)
ne = Point(lng=-122.3, lat=37.9)
sf_box = bbox.create(sw, ne)

# Create from list of points
points = [
    Point(lng=177, lat=-18),
    Point(lng=-178, lat=-17),
    Point(lng=179, lat=-19)
]
fiji_box = bbox.from_points(points)

# Dimensions
print(f"Width: {bbox.width(fiji_box)}°")
print(f"Height: {bbox.height(fiji_box)}°")
print(f"Center: {bbox.center(fiji_box)}")

# Point containment
point = Point(lng=178, lat=-18)
print(f"Contains point: {bbox.contains(fiji_box, point)}")

# Box operations
box1 = BBox(west=0, east=10, south=0, north=10)
box2 = BBox(west=5, east=15, south=5, north=15)

print(f"Boxes intersect: {bbox.intersects(box1, box2)}")
intersection = bbox.intersection(box1, box2)
union = bbox.union(box1, box2)

# Expand box to include a point
expanded = bbox.expand(box1, Point(lng=20, lat=20))
print(f"Expanded box: {expanded}")

## Pythonic Property Access

BBox provides convenient properties alongside the functional API.

In [ ]:
from rhodium.bbox import BBox

fiji = BBox(west=177, east=-178, south=-19, north=-16)

# Functional API
from rhodium import bbox
print(f"Width (functional): {bbox.width(fiji)}°")

# Property API (same result)
print(f"Width (property): {fiji.width}°")
print(f"Height: {fiji.height}°")
print(f"Center: {fiji.center_point}")
print(f"Crosses antimeridian: {fiji.crosses_antimeridian}")

## GeoJSON Integration via `__geo_interface__`

Rhodium implements the Python `__geo_interface__` protocol for compatibility with shapely, fiona, geopandas, and other GIS libraries.

In [ ]:
from rhodium.bbox import Point, BBox
import json

# Point to GeoJSON
point = Point(lng=-122.4, lat=37.8)
print("Point GeoJSON:")
print(json.dumps(point.__geo_interface__, indent=2))

# BBox to GeoJSON Polygon
box = BBox(west=0, east=10, south=0, north=10)
print("\nBBox GeoJSON:")
print(json.dumps(box.__geo_interface__, indent=2))

## Integration with Shapely (Optional)

If you have shapely installed, rhodium objects work seamlessly with it.

In [ ]:
# This cell requires shapely - uncomment to try
# !pip install shapely

# from shapely.geometry import shape
# from rhodium.bbox import Point, BBox

# # Convert rhodium Point to shapely Point
# rh_point = Point(lng=-122.4, lat=37.8)
# sh_point = shape(rh_point.__geo_interface__)
# print(f"Shapely Point: {sh_point}")

# # Convert rhodium BBox to shapely Polygon
# rh_box = BBox(west=0, east=10, south=0, north=10)
# sh_polygon = shape(rh_box.__geo_interface__)
# print(f"Shapely Polygon area: {sh_polygon.area}")

## Performance Characteristics

Rhodium is optimized for correctness, not bulk processing.

In [ ]:
import time
from rhodium import bearing, lng, lat, bbox
from rhodium.bbox import Point, BBox

# Benchmark bearing operations
iterations = 100_000

start = time.perf_counter()
for _ in range(iterations):
    bearing.normalize(365.5)
elapsed = time.perf_counter() - start
print(f"bearing.normalize(): {elapsed/iterations*1e6:.2f} µs per operation ({iterations/elapsed:,.0f} ops/sec)")

start = time.perf_counter()
for _ in range(iterations):
    bearing.diff(350, 10)
elapsed = time.perf_counter() - start
print(f"bearing.diff(): {elapsed/iterations*1e6:.2f} µs per operation ({iterations/elapsed:,.0f} ops/sec)")

# Benchmark bbox operations
box = BBox(west=0, east=10, south=0, north=10)
point = Point(lng=5, lat=5)

start = time.perf_counter()
for _ in range(iterations):
    bbox.contains(box, point)
elapsed = time.perf_counter() - start
print(f"bbox.contains(): {elapsed/iterations*1e6:.2f} µs per operation ({iterations/elapsed:,.0f} ops/sec)")

## Error Handling

Rhodium validates inputs and provides clear error messages.

In [ ]:
from rhodium.bbox import Point, BBox
from rhodium import InvalidLatitudeError, InvalidLongitudeError, InvalidBBoxError

# Invalid latitude
try:
    Point(lng=0, lat=100)  # Latitude must be in [-90, 90]
except InvalidLatitudeError as e:
    print(f"❌ {e}")

# Invalid longitude
try:
    Point(lng=200, lat=0)  # Longitude must be in [-180, 180]
except InvalidLongitudeError as e:
    print(f"❌ {e}")

# Invalid bounding box
try:
    BBox(west=0, east=10, south=50, north=10)  # South cannot be > North
except InvalidBBoxError as e:
    print(f"❌ {e}")

## Real-World Use Cases

### 1. Aviation: Flight Path Calculations

In [ ]:
from rhodium import bearing

# Aircraft turning from 355° to 5° (crossing north)
current_heading = 355
target_heading = 5

turn_angle = bearing.diff(current_heading, target_heading)
print(f"Turn angle: {turn_angle}°")
print(f"Direction: {'right' if turn_angle > 0 else 'left'}")
print(f"Magnitude: {abs(turn_angle)}°")

### 2. Maritime: Pacific Ocean Routes

In [ ]:
from rhodium import lng

# Ship route from Tokyo to Los Angeles
tokyo_lng = 139.7
la_lng = -118.2

# Midpoint for refueling stop
midpoint = lng.mean([tokyo_lng, la_lng])
print(f"Refueling point longitude: {midpoint:.1f}° (in Pacific Ocean)")

# Total distance in longitude degrees
distance = abs(lng.diff(tokyo_lng, la_lng))
print(f"Longitude distance: {distance:.1f}°")

### 3. GIS: Bounding Box for Aleutian Islands

In [ ]:
from rhodium import bbox
from rhodium.bbox import Point, BBox

# Aleutian Islands span the antimeridian
aleutians = BBox(west=165, east=-163, south=51, north=55)

# Check if points are in the Aleutians
attu_island = Point(lng=173.0, lat=52.9)  # Westernmost point of Alaska
unalaska = Point(lng=-166.5, lat=53.9)    # Dutch Harbor

print(f"Attu Island in bbox: {bbox.contains(aleutians, attu_island)}")
print(f"Unalaska in bbox: {bbox.contains(aleutians, unalaska)}")
print(f"Bbox width: {bbox.width(aleutians):.1f}°")
print(f"Bbox center: {bbox.center(aleutians)}")

## Learn More

- **GitHub**: [github.com/marszdf/rhodium](https://github.com/marszdf/rhodium)
- **PyPI**: `pip install elemental-rhodium`
- **Documentation**: See README.md for full API reference
- **Issues/Questions**: [GitHub Issues](https://github.com/marszdf/rhodium/issues)

## Summary

✅ **Use rhodium when**:
- Working with bearings, headings, or compass directions
- Handling longitude coordinates near ±180° (Pacific Ocean, Arctic, etc.)
- Creating bounding boxes that cross the antimeridian
- Averaging or interpolating circular quantities
- Integrating with Python GIS tools (shapely, geopandas)

❌ **Don't use rhodium for**:
- Geodesic distance calculations (use `pyproj` or `geopy`)
- Point-in-polygon tests (use `shapely`)
- Bulk processing 100k+ coordinates (use `numpy` + custom logic)
- Coordinate system transformations (use `pyproj`)